In [16]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import re

In [28]:
# VADER 사전 다운로드 (최초 한 번만 실행)
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\82102\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [29]:
# X_data 디렉토리의 모든 user_CSV 파일들을 병합
import glob
import os
from datetime import datetime

# 모든 user_ CSV 파일 찾기
csv_files = glob.glob("user_*.csv")
print(f"발견된 모든 CSV 파일: {csv_files}")
print(f"총 {len(csv_files)}개의 파일을 병합합니다.")

# 모든 CSV 파일들을 병합
dfs = []
for file in csv_files:
    if os.path.exists(file):
        temp_df = pd.read_csv(file)
        # 파일명에서 사용자명 추출하여 컬럼 추가
        username = file.replace("user_", "").replace("_tweets.csv", "")
        temp_df['username'] = username
        dfs.append(temp_df)
        print(f"{file} 읽기 완료 - {len(temp_df)}개 행")

# 모든 데이터프레임 병합
if dfs:
    df = pd.concat(dfs, ignore_index=True)
    print(f"🎉 총 {len(df)}개 행이 병합되었습니다!")
    print(f"📊 포함된 사용자 수: {len(df['username'].unique())}")
else:
    print("읽을 수 있는 파일이 없습니다.")

발견된 모든 CSV 파일: ['user_@Ajay_Bagga_tweets.csv', 'user_@BillAckman_tweets.csv', 'user_@CathieDWood_tweets.csv', 'user_@elonmusk_tweets.csv', 'user_@JDVance_tweets.csv', 'user_@LizAnnSonders_tweets.csv', 'user_@marcorubio_tweets.csv', 'user_@michaelbatnick_tweets.csv', 'user_@RayDalio_tweets.csv', 'user_@SecScottBessent_tweets.csv', 'user_@sundarpichai_tweets.csv', 'user_@tim_cook_tweets.csv', 'user_@WhiteHouse_tweets.csv']
총 13개의 파일을 병합합니다.
user_@Ajay_Bagga_tweets.csv 읽기 완료 - 530개 행
user_@BillAckman_tweets.csv 읽기 완료 - 800개 행
user_@CathieDWood_tweets.csv 읽기 완료 - 670개 행
user_@elonmusk_tweets.csv 읽기 완료 - 764개 행
user_@JDVance_tweets.csv 읽기 완료 - 701개 행
user_@LizAnnSonders_tweets.csv 읽기 완료 - 673개 행
user_@marcorubio_tweets.csv 읽기 완료 - 806개 행
user_@michaelbatnick_tweets.csv 읽기 완료 - 760개 행
user_@RayDalio_tweets.csv 읽기 완료 - 725개 행
user_@SecScottBessent_tweets.csv 읽기 완료 - 255개 행
user_@sundarpichai_tweets.csv 읽기 완료 - 585개 행
user_@tim_cook_tweets.csv 읽기 완료 - 838개 행
user_@WhiteHouse_tweets.csv 읽기 완료 -

In [30]:
# URL 제거 함수
def remove_urls(text):
    return re.sub(r'https?://\S+', '', text).strip()

In [31]:
# URL 제거 적용
df['full_text'] = df['full_text'].apply(remove_urls)

In [32]:
# 빈 문자열 → NaN
df['full_text'].replace('', pd.NA, inplace=True)

C:\Users\82102\AppData\Local\Temp\ipykernel_36128\1257741985.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['full_text'].replace('', pd.NA, inplace=True)


In [33]:
# NaN 제거 (필요 시)
df.dropna(subset=['full_text'], inplace=True)

In [34]:
# created_at 컬럼 날짜 형식 변환 함수
def convert_date_format(date_str):
    try:
        # "Mon Jun 16 02:50:54 +0000 2025" 형식을 파싱
        dt = pd.to_datetime(date_str, format='%a %b %d %H:%M:%S %z %Y')
        # "2025-06-13 23:00:00" 형식으로 변환 (timezone 제거)
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except:
        return date_str

# created_at 컬럼 형식 변환
print("날짜 형식 변환 중...")
df['created_at'] = df['created_at'].apply(convert_date_format)

# created_at을 datetime 타입으로 변환
df['created_at'] = pd.to_datetime(df['created_at'])

# created_at 기준으로 내림차순 정렬 (최신 순)
df = df.sort_values(by='created_at', ascending=False).reset_index(drop=True)

print("날짜 형식 변환 및 정렬 완료!")
print(f"날짜 범위: {df['created_at'].min()} ~ {df['created_at'].max()}")
print("\n결과 확인:")
print(df.head())

날짜 형식 변환 중...
날짜 형식 변환 및 정렬 완료!
날짜 범위: 2020-01-17 01:05:39 ~ 2025-06-16 04:36:37

결과 확인:
           created_at                                          full_text  \
0 2025-06-16 04:36:37  1. Iran produces around 3.3mn barrels per day ...   
1 2025-06-16 03:40:49             Today at Apple. #F1TheMovie #Severance   
2 2025-06-16 03:04:38  Two countries, separated by 700 kms from each ...   
3 2025-06-16 03:01:08  BREAKING: Iranian opposition Telegram channels...   
4 2025-06-16 02:50:54  26 now.   Note the swing due east at the edge ...   

      username  
0  @Ajay_Bagga  
1    @tim_cook  
2  @Ajay_Bagga  
3  @BillAckman  
4  @BillAckman  


In [35]:
# SentimentIntensityAnalyzer 초기화
sia = SentimentIntensityAnalyzer()

In [36]:
# 감성 분석 함수
def analyze_sentiment(text):
    scores = sia.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        sentiment = 'positive'
    elif compound <= -0.05:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    return pd.Series([sentiment, scores['neg'], scores['neu'], scores['pos']])

In [37]:
# 감성 분석 결과 적용 및 컬럼 추가
df[['sentiment', 'neg', 'neu', 'pos']] = df['full_text'].apply(analyze_sentiment)

In [38]:
# 필요한 컬럼만 추출 (username 컬럼도 포함)
df = df[['created_at', 'full_text', 'username', 'sentiment', 'neg', 'neu', 'pos']]

In [40]:
# 결과 저장
output_filename = "merged_tweets_with_sentiment.csv"
df.to_csv(output_filename, index=False)
print(f"결과가 {output_filename}에 저장되었습니다.")
print(f"총 {len(df)}개의 트윗이 처리되었습니다.")
print(f"포함된 사용자: {sorted(df['username'].unique())}")
print(f"날짜 범위: {df['created_at'].min()} ~ {df['created_at'].max()}")

결과가 merged_tweets_with_sentiment.csv에 저장되었습니다.
총 8594개의 트윗이 처리되었습니다.
포함된 사용자: ['@Ajay_Bagga', '@BillAckman', '@CathieDWood', '@JDVance', '@LizAnnSonders', '@RayDalio', '@SecScottBessent', '@WhiteHouse', '@elonmusk', '@marcorubio', '@michaelbatnick', '@sundarpichai', '@tim_cook']
날짜 범위: 2020-01-17 01:05:39 ~ 2025-06-16 04:36:37
